In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import pymc as pm

In [2]:
df = pl.read_csv("gambia.csv")
df

x,y,pos,age,netuse,treated,green,phc
f64,i64,i64,i64,i64,i64,f64,i64
349631.3,1458055,1,1783,0,0,40.85,1
349631.3,1458055,0,404,1,0,40.85,1
349631.3,1458055,0,452,1,0,40.85,1
349631.3,1458055,1,566,1,0,40.85,1
349631.3,1458055,0,598,1,0,40.85,1
…,…,…,…,…,…,…,…
622086.1,1474011,1,1705,1,0,50.1,1
622086.1,1474011,1,1704,1,0,50.1,1
622086.1,1474011,1,1733,0,0,50.1,1


In [3]:
y = df.get_column("pos").to_numpy()
X = df.select("age", "netuse", "treated", "green", "phc").to_numpy()
y

array([1, 0, 0, ..., 1, 0, 0], shape=(2035,))

In [4]:
X_std = (X - X.mean(0)) / X.std(0)
X_std

array([[ 1.65055383, -1.56872061, -0.61692545, -0.84646886,  0.6804236 ],
       [-1.58918131,  0.63746214, -0.61692545, -0.84646886,  0.6804236 ],
       [-1.47641315,  0.63746214, -0.61692545, -0.84646886,  0.6804236 ],
       ...,
       [ 1.533087  , -1.56872061, -0.61692545,  0.44666349,  0.6804236 ],
       [ 1.74922597, -1.56872061, -0.61692545,  0.44666349,  0.6804236 ],
       [ 1.77506867,  0.63746214, -0.61692545,  0.44666349,  0.6804236 ]],
      shape=(2035, 5))

In [80]:
import numpy as np
from scipy import stats
from scipy.optimize import minimize, minimize_scalar
from scipy.special import beta

def expit(x):
    return 1 / (1 + np.exp(-x))


def dgbetapr(x, a, b, c, d):
    return c * (x / d) ** (a * c - 1) * (1 + (x / d) ** c) ** (-a - b) / (d * beta(a, b))


def W_to_R2_scalar(x, b0):
    # Equation 3 ?
    K = 1000
    p_grid = np.linspace(1, K - 1, num=K - 1) / K

    probs = expit(stats.norm(loc=b0, scale=x ** 0.5).ppf(p_grid))

    # Equations 11 and 12? Maybe 13 I think?
    M = np.sum(probs ** 2) / (K - 1) - (np.sum(probs) / (K - 1)) ** 2
    V = np.sum(probs * (1 - probs)) / (K - 1)

    return M / (M + V)


def W_to_R2(W, b0):
    return np.array([W_to_R2_scalar(w, b0) for w in np.atleast_1d(W)])


def dw(w, a, b, b0):
    # Density function of W
    # Computes values of the density function of W induced by a Beta(a, b) prior on R2.
    delta = 0.001
    diff = pw(w=w + delta, a=a, b=b, b0=b0) - pw(w=w - delta, a=a, b=b, b0=b0)
    return diff / (2 * delta)


def pw(w, a, b, b0):
    # Cumulative Distribution Function (CDF) of W
    # Computes values of the CDF of W induced by a Beta(a, b) prior on R2.
    R2 = W_to_R2(w, b0)
    return stats.beta.cdf(R2, a=a, b=b)


def qw(p, a, b, b0):
    # Quantile Function of W
    # Computes the quantiles of W induced by a Beta(a,b) prior on R2.
    def inner(p):
        def distance(logw):
            return (pw(w=np.exp(logw), a=a, b=b, b0=b0) - p) ** 2

        output = minimize_scalar(
            distance,
            bounds=(np.log(1/100000), np.log(100000)),
            method="bounded"
        ).x

        return np.exp(output)

    return np.array([inner(p_i) for p_i in np.atleast_1d(p)])


def WGBP(a=1, b=1, b0=0, lam=0.25, x0=np.ones(4), method="Powell"):
    tau = np.linspace(0.01, 0.99, num=100)
    w = qw(p=tau, a=a, b=b, b0=b0).flatten()
    py = dw(w=w, a=a, b=b, b0=b0).flatten()


    def distance_fun(log_params):
        params = np.exp(log_params)
        px = dgbetapr(w, *params)
        log_target = np.log(np.array([a, b, 1, 1]))

        penalty = lam * np.sum((log_params - log_target) ** 2)
        distance = np.sum((1 - px / py) ** 2) + penalty
        return distance

    result = minimize(distance_fun, x0=np.log(np.ones(4)), method=method)

    if result.success:
        return np.exp(result.x)

    raise Exception("Minimization didn't converge")

params = WGBP(a=1, b=1, b0=0, lam=0.25)
params

array([1.47463048, 0.66712989, 0.76940629, 1.67539416])

In [76]:
np.exp(m.x)

array([1.47463048, 0.66712989, 0.76940629, 1.67539416])

In [ ]:
params = np.array([1.4722786, 0.6494361, 0.7887448, 1.6711417])

In [ ]:
coords = {
    "predictor": ["age", "netuse", "treated", "green", "phc"],
    "__obs__": np.arange(X_std.shape[0])
}

with pm.Model(coords=coords) as model:
    V = pm.Beta("V", alpha=params[0], beta=params[1])
    W = pm.Deterministic("W", (V / (1 - V)) ** (1 / params[2]) * params[3])
    d = pm.Dirichlet("d", a=np.ones(5), dims="predictor")

    beta0 = pm.Normal("beta0", mu=0, sigma=3 ** 0.5)
    beta = pm.Normal("beta", mu=0, sigma=(W * d) ** 0.5, dims="predictor")

    eta = pm.Deterministic("eta", beta0 + X_std @ beta, dims="__obs__")
    p = pm.Deterministic("p", pm.math.sigmoid(eta), dims="__obs__")
    pm.Bernoulli("y", p=p, observed=y, dims="__obs__")

model.to_graphviz()

In [ ]:
with model:
    idata = pm.sample(target_accept=0.95)

In [ ]:
az.plot_trace(idata, var_names=["V", "W", "d", "beta0", "beta"], backend_kwargs={"layout": "constrained"});